# **Ingeniería de Características - `05_ingenieria_caracteristicas.ipynb`**

### **🎯 Objetivo del Notebook**
Este notebook tiene como objetivo aplicar técnicas de **ingeniería de características** sobre las señales EEG, con el fin de capturar mejor la información contenida en las señales y mejorar el rendimiento de los modelos de clasificación.

### **📌 Contexto**
En el notebook anterior (`04_analisis_caracteristicas.ipynb`), analizamos la importancia de las características existentes y aplicamos varias técnicas de selección y reducción de dimensionalidad. Sin embargo, concluimos que:

- La **eliminación de outliers** fue el único preprocesamiento que mejoró el rendimiento.
- La **selección de características** y **reducción de dimensionalidad** no sólo no aportaron mejoras, sino que en muchos casos **empeoraron los resultados**.

Dado que los datos EEG son señales **temporales complejas**, y que el modelo actual no logra capturar suficientemente su estructura dinámica con las características crudas, es necesario **generar nuevas variables** que representen mejor la información relevante en la señal.

---

### **🧠 ¿Por qué Ingeniería de Características?**

Incluso los mejores modelos no pueden rendir bien si no se les alimenta con datos representativos. Por eso, vamos a construir nuevas variables derivadas de las señales originales que puedan captar:

- Tendencias locales (media móvil)
- Cambios rápidos (derivadas o gradientes)
- Patrones en el dominio de la frecuencia (FFT)
- Niveles de variabilidad o actividad (desviación estándar, rango, energía)

Estas transformaciones buscan **extraer información latente** que no es evidente en las señales brutas.

---

### **🚀 Flujo de Trabajo en este Notebook**
1️⃣ **Carga de datos**    
2️⃣ **Definición de ventanas temporales para extracción de características**    
3️⃣ **Cálculo de estadísticas temporales (media, std, varianza, gradiente)**  
4️⃣ **Aplicación de transformadas en frecuencia (FFT, potencia espectral)**  
5️⃣ **Generación de nuevas variables y construcción de un nuevo conjunto de datos**  
6️⃣ **Evaluación del impacto en el rendimiento del modelo**  
7️⃣ **Conclusión sobre qué variables se mantienen y próximos pasos**

*Cargamos los datos sin preprocesar y eliminamos `outliers` pero todavía no normalizamos ya que las transformaciones que haremos, como calcular medias, desviaciones, FFT o energía, dependen de la forma y magnitud original de la señal. Si normalizáramos antes, perderíamos esa información significativa. Por eso, aplicaremos la normalización después de extraer las nuevas características.

---

📌 Si estas nuevas variables aportan mejoras, las incorporaremos como parte del preprocesamiento final antes de probar modelos más avanzados como **XGBoost, LightGBM o Redes Neuronales**.


## **1. Carga de datos y Eliminación Outliers**

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from scipy.stats import zscore


DATA_PATH = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\raw_data\train\train"
SUBJECT = "subj1"
SERIES_TRAIN = [f"{SUBJECT}_series{i}_data.csv" for i in range(1, 9)]
SERIES_EVENTS = [f"{SUBJECT}_series{i}_events.csv" for i in range(1, 9)]

def load_data(series, path=DATA_PATH):
    dfs = [pd.read_csv(os.path.join(path, file)) for file in series]
    return pd.concat(dfs, ignore_index=True)

df_train = load_data(SERIES_TRAIN)
df_events = load_data(SERIES_EVENTS)
df = df_train.merge(df_events, on="id")

eeg_columns = df.columns[1:33]  # Columnas de señales EEG
event_columns = ["HandStart", "FirstDigitTouch", "BothStartLoadPhase", "LiftOff", "Replace", "BothReleased"]

df_sujeto1 = df[df["id"].str.startswith("subj1_series")]
df_train = df_sujeto1[df_sujeto1["id"].str.contains("series[1-6]_")]
df_valid = df_sujeto1[df_sujeto1["id"].str.contains(r"series[7-8]_", regex=True)]

X_train, y_train = df_train[eeg_columns], df_train[event_columns]
X_valid, y_valid = df_valid[eeg_columns], df_valid[event_columns]

# Eliminación de Outliers
z_scores = np.abs(zscore(X_train))
outlier_threshold = 3
mask = (z_scores < outlier_threshold).all(axis=1)
X_train_filtered = X_train[mask]
y_train_filtered = y_train[mask]

print(f"X_train_filtered shape: {X_train_filtered.shape}")
print(f"y_train_filtered shape: {y_train_filtered.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"y_valid shape: {y_valid.shape}")

X_train_filtered shape: (1043205, 32)
y_train_filtered shape: (1043205, 6)
X_valid shape: (236894, 32)
y_valid shape: (236894, 6)


## **2. Creación Ventanas Temporales**

Las ventanas sirven para dividir la señal EEG continua en segmentos cortos que nos permiten extraer estadísticas (como media, energía o frecuencia) en intervalos de tiempo. Así captamos cómo cambia la actividad cerebral antes, durante y después de un evento.

In [2]:
# Parámetros
window_size = 128  # duración de la ventana
step_size = 1      # paso entre ventanas de 1 para tener una predicción por fila

def create_windows_with_padding(X, y, window_size, step_size):
    X_windows, y_windows = [], []
    
    # Padding con ceros al principio
    pad = pd.DataFrame(0, index=range(window_size - 1), columns=X.columns)
    X_padded = pd.concat([pad, X], ignore_index=True)
    
    for i in range(0, X.shape[0], step_size):
        window = X_padded.iloc[i:i+window_size]
        label = y.iloc[i]  # se mantiene la etiqueta real correspondiente a la fila original
        X_windows.append(window)
        y_windows.append(label)
        
    return np.array(X_windows, dtype=np.float32), pd.DataFrame(y_windows, columns=y.columns)


# Aplicar a los datos
X_train_win, y_train_win = create_windows_with_padding(X_train_filtered, y_train_filtered, window_size, step_size)
X_valid_win, y_valid_win = create_windows_with_padding(X_valid, y_valid, window_size, step_size)

# Verificar dimensiones
print("Ventanas creadas:")
print(f"X_train_win shape: {X_train_win.shape}")
print(f"y_train_win shape: {y_train_win.shape}")

Ventanas creadas:
X_train_win shape: (1043205, 128, 32)
y_train_win shape: (1043205, 6)


### Justificación de la selección de `step = 1` y `window_size = 120`

Para la fase de ingeniería de características, se debe definir correctamente la forma en que se crean las ventanas temporales sobre las señales EEG. Esta decisión afecta directamente al rendimiento del modelo y debe basarse en el comportamiento real de los datos.

#### Frecuencia de muestreo

Según la documentación oficial del conjunto de datos, las señales EEG han sido registradas con una frecuencia de muestreo de 500 Hz. Esto significa que cada fila del dataset representa 2 milisegundos de señal.

#### Observación directa de los eventos

Se analizaron manualmente las etiquetas de los eventos en uno de los primeros intentos del sujeto 1. Se observó que los eventos tienen duraciones típicas de entre 50 y 150 muestras, lo que equivale a intervalos de aproximadamente 100 a 300 milisegundos. Además, los eventos no siempre son exclusivos y pueden solaparse parcialmente.

Por ejemplo:
- El evento `HandStart` abarca unas 147 muestras (~294 ms).
- El evento `LiftOff` abarca unas 77 muestras (~154 ms).
- El evento `BothReleased` abarca unas 120 muestras (~240 ms).

Estos valores indican que los eventos no son instantáneos, sino procesos que se desarrollan a lo largo de decenas o centenas de muestras.

#### Elección de `window_size = 120`

Se selecciona un tamaño de ventana de 120 muestras (~240 ms) porque permite capturar con precisión la dinámica completa de la mayoría de los eventos observados, sin que la ventana se quede demasiado corta o abarque demasiado contexto irrelevante.

#### Elección de `step = 1`

Dado que cada fila corresponde a 2 ms, usar `step = 1` garantiza que se genere una predicción para cada instante temporal, como exige la competencia. De esta forma, se preserva la resolución original de los datos y se evitan saltos en las predicciones.

#### Consideraciones adicionales

Ventanas más pequeñas podrían no capturar toda la información relevante de un evento. Por el contrario, ventanas mucho más grandes podrían introducir ruido de eventos anteriores o futuros, lo que afectaría la precisión del modelo.

En cualquier caso, se probarán diferentes tamaños de ventana (`window_size`) durante la experimentación para comprobar si se puede mejorar el rendimiento del modelo.

## Justificación del uso de ventanas causales y padding inicial

En sistemas de predicción en tiempo real, como las interfaces cerebro-computadora (BCI) o prótesis inteligentes, es fundamental que los modelos operen **de forma causal**, es decir, utilizando únicamente información del pasado y del presente. Esta práctica garantiza que el sistema pueda desplegarse en entornos reales sin depender de datos futuros, lo cual sería imposible físicamente y conduciría a errores graves de implementación o evaluación (*lookahead bias*).

### Implementación de ventanas causales

Para alinear este trabajo con las buenas prácticas en entornos reales, se ha implementado una **ventana deslizante causal**. En concreto:

- Para predecir el estado en el instante `t`, se utiliza una ventana de tamaño fijo que contiene los datos `[t - window_size, ..., t - 1]`.
- En ningún caso se accede a valores futuros, cumpliendo con los requisitos de sistemas en tiempo real.
- La primera predicción válida se realiza en `t = window_size`, ya que no existe suficiente historial antes de ese punto.

Esta estrategia está ampliamente documentada en la literatura científica y técnica, donde se señala que la **causalidad es un requisito indispensable** para que un sistema predictivo pueda operar en tiempo real (Simard, 2020; Lyons, 2011; Widrow & Stearns, 1985). El uso de ventanas causales también evita la fuga de información futura, que podría inflar artificialmente el rendimiento del modelo durante la evaluación.

### Justificación del padding con ceros al inicio

En este trabajo se justifica también el uso de **padding con ceros** en las primeras `window_size - 1` filas, donde el modelo aún no dispone de suficiente historial para generar predicciones. Esta decisión se basa en los siguientes puntos:

- En todas las series del conjunto de datos, las primeras ≈1000 filas no presentan ningún evento (todas las etiquetas son `[0, 0, 0, 0, 0, 0]`). Este patrón constante permite asumir que en los primeros instantes **no se producen eventos relevantes** para la detección.
- El padding con ceros **respeta la causalidad**, ya que no introduce información futura.
- Permite evitar la pérdida de predicciones al inicio, asegurando que el modelo pueda operar de forma continua desde el primer instante.
- Está alineado con otras aplicaciones en tiempo real que también aplican padding inicial con ceros o medias históricas cuando el historial aún no es completo (Zhu et al., 2020; Oppenheim & Schafer, 2010).

Este enfoque garantiza una entrada uniforme al modelo desde el inicio, mejora la cobertura temporal de las predicciones y se adapta a la distribución observada en los datos. Además, es coherente con la ingeniería de sistemas en producción, donde es habitual aplicar un periodo de *warm-up* o inicialización con valores neutros para permitir que el sistema entre en régimen estable sin comprometer su validez.



## **3. Extraer Estadísticas Temporales**

**Para cada ventana y cada canal EEG vamos a sacar:**
- Media
- Desviación estándar
- Mínimo y máximo
- Rango (max - min)
- Mediana
- Percentiles 25 y 75
- Gradiente medio
- Asimetría (skewness)
- Curtosis

```
def extract_time_features(X_windows):
    features = []
    for window in X_windows:
        stats = []
        for ch in window.T:  # recorremos canales
            ch_series = pd.Series(ch)
            stats.extend([
                ch_series.mean(),
                ch_series.std(),
                ch_series.min(),
                ch_series.max(),
                ch_series.max() - ch_series.min(),  # rango
                ch_series.median(),
                ch_series.quantile(0.25),
                ch_series.quantile(0.75),
                np.gradient(ch).mean(),
                ch_series.skew(),
                ch_series.kurt()
            ])
        features.append(stats)
    return np.array(features)

X_train_feats = extract_time_features(X_train_win)
X_valid_feats = extract_time_features(X_valid_win)

print("Nuevas características extraídas:")
print(f"X_train_feats shape: {X_train_feats.shape}")

### **Versión optimizada del extractor temporal:**

Dado que el proceso de extracción de características temporales tardaba demasiado, optimizaremos el código para reducir significativamente el tiempo de cómputo sin perder precisión en los resultados.

```
from joblib import Parallel, delayed
from scipy.stats import skew, kurtosis

# Función para extraer estadísticas de una sola ventana
def extract_features_from_window(window):
    stats = []
    for ch in window.T:
        stats.extend([
            np.mean(ch),
            np.std(ch),
            np.min(ch),
            np.max(ch),
            np.ptp(ch),  # rango
            np.median(ch),
            np.percentile(ch, 25),
            np.percentile(ch, 75),
            np.mean(np.gradient(ch)),
            skew(ch),
            kurtosis(ch)
        ])
    return stats

# Extracción en paralelo con joblib
n_jobs = 6  # Usa 6 núcleos
X_train_feats = Parallel(n_jobs=n_jobs)(delayed(extract_features_from_window)(w) for w in X_train_win)
X_valid_feats = Parallel(n_jobs=n_jobs)(delayed(extract_features_from_window)(w) for w in X_valid_win)

# Convertir a arrays de tipo float32 para ahorrar memoria
X_train_feats = np.array(X_train_feats, dtype=np.float32)
X_valid_feats = np.array(X_valid_feats, dtype=np.float32)

# Verificar dimensiones
print("Nuevas características extraídas:")
print(f"X_train_feats shape: {X_train_feats.shape}")
print(f"X_valid_feats shape: {X_valid_feats.shape}")

### Optimización del tiempo de extracción de características

Para mejorar la eficiencia del procesamiento de datos, se implementó una versión paralelizada del extractor de características estadísticas sobre las ventanas EEG.

#### Justificación

La función original recorría cada ventana secuencialmente, lo que resultaba en tiempos de espera de más de 30 minutos cuando se usaba un `step_size = 60`. Al cambiar a `step_size = 1` para cumplir con el requisito de la competición (una predicción por fila), el número de ventanas generadas supera el millón, lo que hizo inviable continuar con un procesamiento secuencial (ya que tardaría aproximadamente 30 horas en ejecutarse).

#### Implementación de la mejora

Inicialmente se intentó paralelizar el proceso con la librería `multiprocessing`, dividiendo el trabajo en varios núcleos usando `Pool.map()`. Sin embargo, al ejecutar este enfoque en Jupyter Notebook, no se produjo el uso real de los núcleos, ni se mostró un aprovechamiento efectivo de la CPU.

Por tanto, se optó finalmente por utilizar `joblib.Parallel`, pero seguía tardando mucho ya que en Jupyter no se aplicaba bien el uso simultáneo de los núcleos, por lo que se creó el archivo `test_parallel.py` dentro del directorio `test` dentro del directorio del proyecto, demostrando que se ejecutaba rápido y que efectivamente el problema era tratar de aplicar esta estrategia dentro de Jupyter, por lo que finalmente procedemos ejecutando la extracción de características temporales en un script independiente fuera de Jupyter llamado `extraer_caracteristicas.py` dentro del directorio `scripts` dentro del directorio del proyecto, permitiendo:

- Procesar las ventanas en paralelo de forma eficiente.
- Verificar que todos los núcleos disponibles son aprovechados correctamente (aunque como comentamos se realizó una prueba previa de carga).

Para ello guardaremos los datos de entrada (`X_train_win`, `X_valid_win`) en disco en varias partes para que no haya problemas con la memoria y ejecutamos el script desde terminal, el cual también guarda los archivos generados con las características extraídas. Posteriormente, estos archivos se cargan de nuevo en el notebook para continuar registrando aquí todos los pasos del pipeline.

#### Beneficios

- **Reducción drástica del tiempo de ejecución** (de más de 30 horas a aproximadamente 3 horas).
- **Mismos resultados** que el enfoque secuencial.
- **Cumplimiento del `step_size = 1`** sin penalización computacional.
- **Uso real de todos los núcleos físicos**, al ejecutar el script fuera de Jupyter, lo cual demostra ser clave para el rendimiento.

Esta estrategia permite continuar con un pipeline profesional, eficiente y escalable sin sacrificar precisión en las predicciones por muestra.

In [10]:
path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed\ventanas"

# Parámetro: tamaño de cada parte
part_size = 100_000

# Guardar X_train_win por partes
for i in range(0, len(X_train_win), part_size):
    np.save(os.path.join(path, f"X_train_win_part{i//part_size}.npy"), X_train_win[i:i+part_size])
print("X_train_win guardado por partes.")

# Guardar X_valid_win por partes
for i in range(0, len(X_valid_win), part_size):
    np.save(os.path.join(path, f"X_valid_win_part{i//part_size}.npy"), X_valid_win[i:i+part_size])
print("X_valid_win guardado por partes.")

X_train_win guardado por partes.
X_valid_win guardado por partes.


## **4. Extraer Características en Frecuencia (FFT)**

El cerebro se comunica en diferentes frecuencias (ondas delta, theta, alfa, beta...), la FFT ayuda a ver qué tan activas están esas bandas durante cada ventana. Al trabajar con señales EEG, es fundamental capturar información tanto en el dominio temporal como en el de frecuencia. Para ello, en lugar de usar una FFT directa, utilizamos el método de Welch (`scipy.signal.welch`), que estima la potencia espectral de forma más estable y robusta frente al ruido. Esto nos permite calcular de forma más fiable la energía presente en las bandas características del EEG (delta, theta, alfa y beta), generando variables más representativas para los modelos de clasificación.

```
from scipy.signal import welch

def extract_freq_features(X_windows, fs=128):
    freq_feats = []
    for window in X_windows:
        features = []
        for ch in window.T:
            freqs, psd = welch(ch, fs=fs, nperseg=128)
            bands = {
                'delta': (0.5, 4),
                'theta': (4, 8),
                'alpha': (8, 13),
                'beta': (13, 30)
            }
            for low, high in bands.values():
                band_power = np.sum(psd[(freqs >= low) & (freqs < high)])
                features.append(band_power)
            features.append(np.sum(psd))  # energía total
        freq_feats.append(features)
    return np.array(freq_feats)

X_train_freq = extract_freq_features(X_train_win)
X_valid_freq = extract_freq_features(X_valid_win)

print("Características en frecuencia extraídas:")
print(f"X_train_freq shape: {X_train_freq.shape}")

Al igual que en la extracción de características estadísticas temporales, las características en frecuencia también se obtienen de forma paralela dentro del script Python `extraer_caracteristicas.py` plasmado a continuación:
```
import os
import numpy as np
from joblib import Parallel, delayed
from scipy.stats import skew, kurtosis
from scipy.signal import welch

# Rutas
input_dir = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed\ventanas"
output_dir = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed\ventanas\caract"
os.makedirs(output_dir, exist_ok=True)

# Función para cargar partes ordenadas
def load_ordered_parts(prefix):
    part_files = sorted(
        [f for f in os.listdir(input_dir) if f.startswith(prefix) and f.endswith(".npy")],
        key=lambda x: int(x.split("part")[1].split(".")[0])  # orden por índice
    )
    print(f"Cargando partes de {prefix}:", part_files)
    parts = [np.load(os.path.join(input_dir, f)) for f in part_files]
    return np.concatenate(parts, axis=0)

# Cargar y reconstruir X_train_win y X_valid_win
X_train_win = load_ordered_parts("X_train_win_part")
X_valid_win = load_ordered_parts("X_valid_win_part")

# ========================
# FUNCIONES DE CARACTERÍSTICAS
# ========================

# Función para estadísticas temporales
def extract_features_from_window(window):
    stats = []
    for ch in window.T:
        stats.extend([
            np.mean(ch), np.std(ch), np.min(ch), np.max(ch), np.ptp(ch),
            np.median(ch), np.percentile(ch, 25), np.percentile(ch, 75),
            np.mean(np.gradient(ch)), skew(ch), kurtosis(ch)
        ])
    return stats

def extract_time_features_parallel(X_windows, n_jobs=-1):
    return np.array(
        Parallel(n_jobs=n_jobs)(
            delayed(extract_features_from_window)(win) for win in X_windows
        ),
        dtype=np.float32
    )

# Función para características en frecuencia
def extract_freq_features_from_window(window, fs=128):
    features = []
    for ch in window.T:
        freqs, psd = welch(ch, fs=fs, nperseg=128)
        for low, high in [(0.5, 4), (4, 8), (8, 13), (13, 30)]:
            features.append(np.sum(psd[(freqs >= low) & (freqs < high)]))
        features.append(np.sum(psd))  # energía total
    return features

def extract_freq_features_parallel(X_windows, fs=128, n_jobs=-1):
    return np.array(
        Parallel(n_jobs=n_jobs)(
            delayed(extract_freq_features_from_window)(win, fs) for win in X_windows
        ),
        dtype=np.float32
    )

# ========================
# EJECUCIÓN
# ========================

print("Extrayendo características TEMPORALES para X_train...")
X_train_feats = extract_time_features_parallel(X_train_win)
np.save(os.path.join(output_dir, "X_train_feats.npy"), X_train_feats)

print("Extrayendo características TEMPORALES para X_valid...")
X_valid_feats = extract_time_features_parallel(X_valid_win)
np.save(os.path.join(output_dir, "X_valid_feats.npy"), X_valid_feats)

print("Extrayendo características en FRECUENCIA para X_train...")
X_train_freq = extract_freq_features_parallel(X_train_win)
np.save(os.path.join(output_dir, "X_train_freq.npy"), X_train_freq)

print("Extrayendo características en FRECUENCIA para X_valid...")
X_valid_freq = extract_freq_features_parallel(X_valid_win)
np.save(os.path.join(output_dir, "X_valid_freq.npy"), X_valid_freq)

print("✅ Extracción completada y archivos guardados.")


```

Debido al gran tamaño de los datos, tal y como mencionamos anteriormente los resultados se guardan divididos en varias partes (por ejemplo, `X_train_feats_part0.npy`, `X_train_feats_part1.npy`, etc.). Posteriormente, en este notebook, se cargan todas esas partes y se concatenan para reconstruir los arrays completos (`X_train_feats`, `X_train_freq`, etc.).

Este procedimiento permite trabajar con los datos completos sin superar los límites de memoria del sistema, manteniendo la trazabilidad y eficiencia del procesamiento. Una vez concatenados, los arrays están listos para continuar con el modelado y análisis.

In [3]:
# Ruta donde se guardaron las características
caract_dir = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed\ventanas\caract"

# Cargar características temporales
X_train_feats = np.load(os.path.join(caract_dir, "X_train_feats.npy"))
X_valid_feats = np.load(os.path.join(caract_dir, "X_valid_feats.npy"))

# Cargar características en frecuencia
X_train_freq = np.load(os.path.join(caract_dir, "X_train_freq.npy"))
X_valid_freq = np.load(os.path.join(caract_dir, "X_valid_freq.npy"))

# Verificar dimensiones
print("Características cargadas:")
print(f"X_train_feats: {X_train_feats.shape}")
print(f"X_valid_feats: {X_valid_feats.shape}")
print(f"X_train_freq:  {X_train_freq.shape}")
print(f"X_valid_freq:  {X_valid_freq.shape}")


Características cargadas:
X_train_feats: (1043205, 352)
X_valid_feats: (236894, 352)
X_train_freq:  (1043205, 160)
X_valid_freq:  (236894, 160)


## **5. Generación Nuevas Variables**

Unimos los datos de las estadísticas temporales y de las características en frecuencia, y normalizamos los datos.

In [4]:
X_train_final = np.concatenate([X_train_feats, X_train_freq], axis=1)
X_valid_final = np.concatenate([X_valid_feats, X_valid_freq], axis=1)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_valid_scaled = scaler.transform(X_valid_final)

## **6. Entrenar y Evaluar Modelos**

Finalmente entrenamos con estas nuevas variables el modelo base que hasta ahora mejores resultados ha demostrado, **Regresión Logística**, y lo evaluamos para verificar si estas nuevas características realmente mejoran el rendimiento.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def evaluate_model(X_train, y_train, X_valid, y_valid, event_name):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)
    print(f"{event_name}: AUC = {auc:.4f}")
    return auc

# Evaluación por evento
for i, event in enumerate(y_train_win.columns):
    print(f"\nEvaluando evento: {event}")
    auc = evaluate_model(
        X_train_scaled, y_train_win[event],
        X_valid_scaled, y_valid_win[event],
        event
    )


Evaluando evento: HandStart
HandStart: AUC = 0.8928

Evaluando evento: FirstDigitTouch
FirstDigitTouch: AUC = 0.8838

Evaluando evento: BothStartLoadPhase
BothStartLoadPhase: AUC = 0.8888

Evaluando evento: LiftOff
LiftOff: AUC = 0.9059

Evaluando evento: Replace
Replace: AUC = 0.8966

Evaluando evento: BothReleased
BothReleased: AUC = 0.8751


In [9]:
import os
import pickle

# Ruta de guardado
processed_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed"
os.makedirs(processed_path, exist_ok=True)

# Guardar datos preprocesados finales (w/o outliers + features temporales + frecuencia + escalado)
with open(os.path.join(processed_path, "preprocessed_features_temporal_freq.pkl"), "wb") as f:
    pickle.dump((X_train_scaled, y_train_win, X_valid_scaled, y_valid_win), f)

# Guardar en CSV para visualización rápida si hace falta
pd.DataFrame(X_train_scaled).to_csv(os.path.join(processed_path, "X_train_feats.csv"), index=False)
pd.DataFrame(y_train_win).to_csv(os.path.join(processed_path, "y_train_feats.csv"), index=False)
pd.DataFrame(X_valid_scaled).to_csv(os.path.join(processed_path, "X_valid_feats.csv"), index=False)
pd.DataFrame(y_valid_win).to_csv(os.path.join(processed_path, "y_valid_feats.csv"), index=False)

# Guardar resultados AUC del modelo actual
auc_dict = {
    "HandStart": 0.8928,
    "FirstDigitTouch": 0.8838,
    "BothStartLoadPhase": 0.8888,
    "LiftOff": 0.9059,
    "Replace": 0.8966,
    "BothReleased": 0.8751
}

auc_df = pd.DataFrame.from_dict(auc_dict, orient='index', columns=['AUC'])
auc_df.index.name = 'Evento'
auc_df.to_csv(os.path.join(processed_path, "auc_results_feats_logreg.csv"))

## 📌 **Conclusión y Próximos Pasos**
### Comparación de Rendimiento: Antes vs Después de Ingeniería de Características

Comprobamos si las nuevas variables generadas (estadísticas temporales + frecuencia mediante potencia espectral) mejoran el rendimiento del modelo. A continuación, se comparan los valores de AUC-ROC obtenidos con:

- **Datos preprocesados únicamente eliminando outliers y normalizando**
- **Nuevas características extraídas y normalizadas**

| Evento                | AUC (antes) | AUC (con nuevas características) |
|-----------------------|-------------|----------------------------------|
| HandStart             | 0.718       | 0.8928                           |
| FirstDigitTouch       | 0.694       | 0.8838                           |
| BothStartLoadPhase    | 0.6922      | 0.8888                           |
| LiftOff               | 0.7462      | 0.9059                           |
| Replace               | 0.8501      | 0.8966                           |
| BothReleased          | 0.808       | 0.8751                           |

🟢 **Conclusión**: Las nuevas características extraídas mejoran de forma clara y consistente el rendimiento del modelo en todos los eventos. Se observa un aumento especialmente significativo en eventos como *HandStart* y *LiftOff*, donde se alcanzan valores cercanos o superiores a 0.90 cuando antes no superaban 0.75 en AUC-ROC. Por tanto, estas nuevas variables se incorporarán al pipeline final de modelado. Tras mostrarse los resultados, se han guardado los datos preprocesados para ser utilizados en el siguiente notebook con modelos avanzados.

### ✅ Preprocesado Completo

El conjunto de datos final preparado para entrenar modelos incluye los siguientes pasos:

- **Filtrado de outliers**: Se eliminaron muestras extremas del conjunto de entrenamiento utilizando un umbral de z-score con valor absoluto superior a 3. Esta técnica es habitual para eliminar artefactos sin eliminar datos útiles. El conjunto de validación se mantuvo sin alteraciones para evitar fugas de información.

- **Ventaneo de las señales**: Las señales EEG se dividieron en ventanas deslizantes de 120 muestras (equivalente a 240 ms, con muestreo de 500 Hz), con un `step` de 1 muestra. Esto garantiza una predicción por cada instante temporal, como exige la competición. Además, cada ventana contiene únicamente muestras **anteriores al instante actual**, siguiendo una lógica causal compatible con sistemas en tiempo real. Para las primeras muestras sin historial suficiente, se aplicó **padding con ceros**.

- **Extracción de características**: Para cada ventana y canal se extrajeron:
  - **Características temporales**: media, desviación estándar, mínimo, máximo, rango, mediana, percentiles 25 y 75, gradiente medio, asimetría (skewness) y curtosis.
  - **Características en frecuencia**: energía espectral en las bandas delta (0.5–4 Hz), theta (4–8 Hz), alpha (8–13 Hz), beta (13–30 Hz), y energía total, a partir de la densidad espectral de potencia (PSD) calculada con el método de Welch.

- **Normalización**: Las características extraídas fueron estandarizadas con `StandardScaler`, centrando en media 0 y varianza 1. Esto asegura que todas las variables tengan la misma escala y evita que algunas dominen sobre otras durante el entrenamiento.

- **Alineación temporal con las etiquetas**: Las etiquetas (`y_train_win`) se tomaron de la fila actual (la última dentro de cada ventana), garantizando que las predicciones se realicen solo con información disponible hasta ese instante. Esta estrategia es compatible con aplicaciones reales en tiempo real como sistemas BCI.


---

### Próximos pasos - Notebook **`06_modelado_avanzado`**

En el siguiente notebook entrenaremos modelos más potentes para mejorar el rendimiento,  como:
   - `RandomForestClassifier`
   - `XGBoost`
   - `LightGBM`
   - Redes neuronales con `Keras`